In [6]:
import pandas as pd
import numpy as np
import requests
import time
from pprint import pprint
import json

from sqlalchemy import create_engine
import pymysql
pymysql.install_as_MySQLdb()

from bs4 import BeautifulSoup
import requests
import pymongo

In [7]:
# *** Screen Scrape tables ***

state_blackfriday_violence_url = "https://www.reviews.org/trends/high-risk-states-for-black-friday-violence/"

blackfriday_death_count_url = "http://blackfridaydeathcount.com"

In [8]:
# Get first table data

state_blackfriday_violence_table = pd.read_html(state_blackfriday_violence_url, header=0)

In [9]:
# Get second table data

blackfriday_death_count_table = pd.read_html(blackfriday_death_count_url, header=0)

In [10]:
# get first table dataframe

state_df = state_blackfriday_violence_table[0]
state_df.head()

,Our ranking,State,Violent Crime ranking,Interest in Black Friday
0,1,ARKANSAS,6,15
1,2,TENNESSEE,3,14
2,3,WEST VIRGINIA,28,1
3,4,NORTH CAROLINA,24,21
4,5,ALABAMA,8,26


In [27]:
# get second table dataframe

bf_death = blackfriday_death_count_table[0]

bf_death.head()

,Unnamed: 0,Unnamed: 1,Deaths,Injuries
0,2018,"Man, 27, shot to death in parking lot in South...",1,-
1,2018,Man wounded in shooting inside N.J. mall at th...,-,1
2,2018,Two people stabbed at Macy's in Destiny USA,-,2
3,2018,Glance at attractive woman sparked Memphis mal...,-,1
4,2018,One dead and 2 wounded— including 12-year-old ...,1,2


In [28]:
# Clean first table data

new_state_df = state_df[['Our ranking','State','Violent Crime ranking']]
new_state_df2=new_state_df.rename(columns={"Our ranking":"ranking","State":"state","Violent Crime ranking":"vio_crime_rank"})
new_state_df2.head()

,ranking,state,vio_crime_rank
0,1,ARKANSAS,6
1,2,TENNESSEE,3
2,3,WEST VIRGINIA,28
3,4,NORTH CAROLINA,24
4,5,ALABAMA,8


In [39]:
# Clean second table data

bf_death_df = bf_death.rename(columns={"Unnamed: 0":"year","Unnamed: 1":"article_title","Deaths":"deaths","Injuries":"injuries"})

bf_death_df2 = bf_death_df.replace('-',0)
#bf_death_df3 = bf_death_df2.replace(':'," ")
#bf_death_df2 = bf_death_df2.article_title.str.replace(':','')


bf_death_df3 = bf_death_df2.dropna()

bf_death_df4 = bf_death_df3.reset_index(drop=True)
bf_death_df4.drop(bf_death_df2.index[47], inplace=True)
bf_death_df5 = bf_death_df4.reset_index(drop=True)
bf_death_df5.head(50)

,year,article_title,deaths,injuries
0,2018,"Man, 27, shot to death in parking lot in South...",1,0
1,2018,Man wounded in shooting inside N.J. mall at th...,0,1
2,2018,Two people stabbed at Macy's in Destiny USA,0,2
3,2018,Glance at attractive woman sparked Memphis mal...,0,1
4,2018,One dead and 2 wounded— including 12-year-old ...,1,2
5,2017,Black Friday altercation in Kmart leaves man w...,0,1
6,2017,Black Friday Missouri mall shooting leaves tee...,0,1
7,2017,"One male shot, one male stabbed at Willowbrook...",0,2
8,2017,"Brawls close Alabama shopping center, one trea...",0,1
9,2016,San Antonio man helped a woman being beaten in...,1,2


In [15]:
# Connect to local Database

rds_connection_string = "root:5663307t@localhost:3306/black_friday_incidents"
engine = create_engine(f'mysql://{rds_connection_string}')


#engine = create_engine(f"mysql://{dbuser}:{dbpasswd}@{dburi}:{dbport}/{dbname}")
#Base.metadata.create_all(engine)
#dbuser = "root"
#dbpasswd = ""
#dburi = "localhost"
#dbport = 3306
#dbname = "SharkSearch"

In [16]:
engine.table_names()

['incident_by_year', 'state_violentcrimes']

In [17]:
# first table transferred to MYSQL

new_state_df2.to_sql(name='state_violentcrimes',con=engine,if_exists='append', index=False)

In [40]:
# second table transferred to MYSQL

bf_death_df5.to_sql(name='incident_by_year',con=engine,if_exists='append', index=False)

In [87]:
# Verify first table data

pd.read_sql_query('select * from state_violentcrimes',con=engine).head()

,ranking,state,vio_crime_rank
0,1,ARKANSAS,6
1,2,TENNESSEE,3
2,3,WEST VIRGINIA,28
3,4,NORTH CAROLINA,24
4,5,ALABAMA,8


In [41]:
# Verify second table data

pd.read_sql_query('select * from incident_by_year',con=engine).head()

,year,article_title,deaths,injuries
0,2018,"Man, 27, shot to death in parking lot in South...",1,0
1,2018,Man wounded in shooting inside N.J. mall at th...,0,1
2,2018,Two people stabbed at Macy's in Destiny USA,0,2
3,2018,Glance at attractive woman sparked Memphis mal...,0,1
4,2018,One dead and 2 wounded— including 12-year-old ...,1,2


In [ ]:
# Dependencies
# ----------------------------------
# Imports the method used for connecting to DBs
from sqlalchemy import create_engine

# Imports the methods needed to abstract classes into tables
from sqlalchemy.ext.declarative import declarative_base

# Allow us to declare column types
from sqlalchemy import Column, Integer, String, Float 

# PyMySQL 
import pymysql
pymysql.install_as_MySQLdb()

In [10]:
blackfriday_death_count_table = pd.read_html(blackfriday_death_count_url)
blackfriday_death_count_table

[                                             0  \
 0                                          NaN   
 1                                         2018   
 2                                         2018   
 3                                         2018   
 4                                         2018   
 5                                         2018   
 6                                         2017   
 7                                         2017   
 8                                         2017   
 9                                         2017   
 10                                        2016   
 11                                        2016   
 12                                        2016   
 13                                        2016   
 14                                        2016   
 15                                        2016   
 16                                        2016   
 17                                        2014   
 18                            

In [ ]:
# **** USING MONGO DB ***********

In [40]:
# Initialize PyMongo to work with MongoDBs
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

In [41]:
# Define database and collection
state_bf_db = client.state_bf_db
collection_state_bf = state_bf_db.items

bf_dc_db = client.bf_deathcount_db
collection_bf_deathcount = bf_dc_db.items

In [42]:
# Retrieve page with the requests module
response1 = requests.get(state_blackfriday_violence_url)
# Create BeautifulSoup object; parse with 'lxml'
soup1 = BeautifulSoup(response1.text, 'lxml')

# Retrieve page with the requests module
response2 = requests.get(blackfriday_death_count_url)
# Create BeautifulSoup object; parse with 'lxml'
soup2 = BeautifulSoup(response2.text, 'lxml')

In [51]:
results = soup1.find_all('tr',class_="default-row")

for result in results:
    
    try:
        tds = result.find_all('td')
        our_ranking = tds[0].text
        
        #state = result.find('td'[1]).text
        state=tds[1].text
        
        #violent_crime_ranking = result.find('td'[2]).text
        violent_crime_ranking=tds[2].text
        
        #interest_in_bf = result.find('td').text
        interest_in_bf=tds[3].text
        
        if (our_ranking and state and violent_crime_ranking and interest_in_bf):
            print("------------------")
            print(our_ranking)
            print(state)
            print(violent_crime_ranking)
            print(interest_in_bf)
            
            post = {
                'rank':our_ranking,
                'state':state,
                'violent_crime_ranking':violent_crime_ranking,
                'interest_in_black_friday':interest_in_bf
            }
            
            collection_state_bf.insert_one(post)
            
    except Exception as e:
        print(e)

------------------
1
ARKANSAS
6
15
------------------
2
TENNESSEE
3
14
------------------
3
WEST VIRGINIA
28
1
------------------
4
NORTH CAROLINA
24
21
------------------
5
ALABAMA
8
26
------------------
6
NEVADA
5
41
------------------
7
CALIFORNIA
15
39
------------------
8
MISSOURI
7
18
------------------
9
ARIZONA
9
7
------------------
10
ILLINOIS
17
22
------------------
11
SOUTH CAROLINA
10
36
------------------
12
WYOMING
41
38
------------------
13
MICHIGAN
14
17
------------------
14
DELAWARE
13
9
------------------
15
NEW JERSEY
43
32
------------------
16
CONNECTICUT
44
27
------------------
17
TEXAS
16
42
------------------
18
NEW YORK
27
45
------------------
19
WASHINGTON
32
25
------------------
20
ALASKA
1
50
------------------
21
GEORGIA
26
31
------------------
22
FLORIDA
20
30
------------------
23
IOWA
34
5
------------------
24
COLORADO
23
35
------------------
25
KANSAS
19
19
------------------
26
INDIANA
21
23
------------------
27
IDAHO
45
3
-----------------

In [52]:
# Display items in MongoDB collection
listings = state_bf_db.items.find()

for listing in listings:
    print(listing)

{'_id': ObjectId('5c17c388484c9e33e4c2d393'), 'rank': '1', 'state': 'ARKANSAS', 'violent_crime_ranking': '6', 'interest_in_black_friday': '15'}
{'_id': ObjectId('5c17c388484c9e33e4c2d394'), 'rank': '2', 'state': 'TENNESSEE', 'violent_crime_ranking': '3', 'interest_in_black_friday': '14'}
{'_id': ObjectId('5c17c388484c9e33e4c2d395'), 'rank': '3', 'state': 'WEST VIRGINIA', 'violent_crime_ranking': '28', 'interest_in_black_friday': '1'}
{'_id': ObjectId('5c17c388484c9e33e4c2d396'), 'rank': '4', 'state': 'NORTH CAROLINA', 'violent_crime_ranking': '24', 'interest_in_black_friday': '21'}
{'_id': ObjectId('5c17c388484c9e33e4c2d397'), 'rank': '5', 'state': 'ALABAMA', 'violent_crime_ranking': '8', 'interest_in_black_friday': '26'}
{'_id': ObjectId('5c17c388484c9e33e4c2d398'), 'rank': '6', 'state': 'NEVADA', 'violent_crime_ranking': '5', 'interest_in_black_friday': '41'}
{'_id': ObjectId('5c17c388484c9e33e4c2d399'), 'rank': '7', 'state': 'CALIFORNIA', 'violent_crime_ranking': '15', 'interest_in_